In [6]:
# Modify the API script to resample data to daily instead of hourly
import requests
import pandas as pd
import datetime as dt

# Define API details
DMI_URL = 'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items'
api_key = '56642815-d535-418b-bda2-30a8a8ef4999'

# Specify the desired start and end time
start_time = pd.Timestamp(2002, 1, 1)
end_time = pd.Timestamp(2023, 12, 31)
datetime_str = start_time.tz_localize('UTC').isoformat() + '/' + end_time.tz_localize('UTC').isoformat()

# Station ID
stationIds = ['06102']

# List of required parameters
parameterIds = [
    "temp_min_past1h", "temp_max_past1h", "temp_mean_past1h",
    "temp_grass_mean_past1h", "temp_soil_min_past1h", "temp_soil_max_past1h", "temp_soil_mean_past1h",
    "humidity_past1h", "pressure", "wind_dir_past1h", "wind_min_past1h",
    "wind_gust_always_past1h", "wind_speed_past1h", "precip_past1h", "precip_dur_past1h",
    "radia_glob_past1h", "sun_last1h_glob"
]

# Fetch data for all parameters
dfs = []
for station in stationIds:
    for parameter in parameterIds:
        # Define API query parameters
        params = {
            'api-key': api_key,
            'datetime': datetime_str,
            'stationId': station,
            'parameterId': parameter,
            'limit': '300000',  # Max limit
        }

        # Send GET request
        response = requests.get(DMI_URL, params=params)

        # Check response status
        if response.status_code == 200:
            json_data = response.json()

            # Extract data into DataFrame
            if 'features' in json_data and json_data['features']:
                dfi = pd.json_normalize(json_data['features'])
                dfi['time'] = pd.to_datetime(dfi['properties.observed'])
                dfi = dfi[['time', 'properties.value', 'properties.stationId', 'properties.parameterId']]
                dfi.columns = [c.replace('properties.', '') for c in dfi.columns]
                dfi = dfi[~dfi.duplicated()]  # Remove duplicates
                dfi = dfi.set_index(['parameterId', 'stationId', 'time'])
                dfi = dfi['value'].unstack(['stationId', 'parameterId'])
                dfs.append(dfi)

# Combine all data into a single DataFrame
if dfs:
    df = pd.concat(dfs, axis='columns').sort_index()

    # Flatten MultiIndex by removing station ID level and keeping only parameter names
    df.columns = df.columns.get_level_values(1)

    # Define aggregation methods for daily resampling
    aggregation_methods = {
        "temp_min_past1h": "min",  # Minimum temperature of the day
        "temp_max_past1h": "max",  # Maximum temperature of the day
        "temp_mean_past1h": "mean",  # Daily mean temperature
        "temp_grass_mean_past1h": "mean",  # Mean grass temperature
        "temp_soil_min_past1h": "min",  # Minimum soil temperature
        "temp_soil_max_past1h": "max",  # Maximum soil temperature
        "temp_soil_mean_past1h": "mean",  # Mean soil temperature
        "humidity_past1h": "mean",  # Average humidity of the day
        "pressure": "mean",  # Mean atmospheric pressure
        "wind_dir_past1h": "mean",  # Average wind direction
        "wind_min_past1h": "min",  # Minimum wind speed
        "wind_gust_always_past1h": "max",  # Maximum wind gust
        "wind_speed_past1h": "mean",  # Mean wind speed
        "precip_past1h": "sum",  # Total daily precipitation
        "precip_dur_past1h": "sum",  # Total minutes of precipitation
        "radia_glob_past1h": "sum",  # Total daily global radiation
        "sun_last1h_glob": "sum",  # Total sunshine duration per day
    }

    # Resample the DataFrame to daily using defined aggregation rules
    df_daily = df.resample("D").agg(aggregation_methods)

    # Save as CSV
    df_daily.to_csv("dmi_weather_data_daily.csv")

else:
    print("No data retrieved. Check API response and station/parameter configuration.")


In [24]:
df_daily.tail() # Show first rows

parameterId,temp_min_past1h,temp_max_past1h,temp_mean_past1h,temp_grass_mean_past1h,temp_soil_min_past1h,temp_soil_max_past1h,temp_soil_mean_past1h,humidity_past1h,pressure,wind_dir_past1h,wind_min_past1h,wind_gust_always_past1h,wind_speed_past1h,precip_past1h,precip_dur_past1h,radia_glob_past1h,sun_last1h_glob
time,,,,,,,,,,,,,,,,,
2023-12-20 00:00:00+00:00,0.8,4.4,3.141667,2.316667,5.1,5.4,5.241667,88.583333,995.043373,252.000000,NaN,12.2,2.541667,0.4,4.0,349.0,111.0
2023-12-21 00:00:00+00:00,1.4,8.7,5.316667,4.637500,5.0,5.3,5.141667,86.083333,975.337324,254.666667,0.7,22.5,5.329167,19.9,159.0,270.0,29.5
2023-12-22 00:00:00+00:00,0.1,4.0,2.395833,1.129167,4.0,5.1,4.533333,66.000000,983.997222,293.416667,0.0,19.1,5.020833,0.6,3.0,749.0,294.5
2023-12-23 00:00:00+00:00,-1.6,3.3,0.833333,-0.250000,3.6,4.1,3.754167,88.000000,993.885417,264.000000,NaN,10.5,2.379167,0.8,4.0,589.0,247.0
2023-12-24 00:00:00+00:00,-0.3,2.1,1.120000,0.700000,3.4,3.6,3.520000,97.700000,992.207273,136.500000,NaN,6.6,1.500000,5.0,38.0,9.0,0.0


In [9]:
# Check for missing values
missing_values = df_daily.isnull().sum()
print("Missing Data Count:\n", missing_values[missing_values > 0])

Missing Data Count:
 parameterId
temp_min_past1h             150
temp_max_past1h             150
temp_mean_past1h            150
temp_grass_mean_past1h      138
temp_soil_min_past1h        138
temp_soil_max_past1h        138
temp_soil_mean_past1h       138
humidity_past1h             160
pressure                   5931
wind_dir_past1h             143
wind_min_past1h            6245
wind_gust_always_past1h     758
wind_speed_past1h           143
dtype: int64


In [10]:
df_daily.count()

parameterId
temp_min_past1h            7878
temp_max_past1h            7878
temp_mean_past1h           7878
temp_grass_mean_past1h     7890
temp_soil_min_past1h       7890
temp_soil_max_past1h       7890
temp_soil_mean_past1h      7890
humidity_past1h            7868
pressure                   2097
wind_dir_past1h            7885
wind_min_past1h            1783
wind_gust_always_past1h    7270
wind_speed_past1h          7885
precip_past1h              8028
precip_dur_past1h          8028
radia_glob_past1h          8028
sun_last1h_glob            8028
dtype: int64

In [11]:
# Define threshold for missing data (e.g., remove columns with >50% missing)
missing_threshold = 0.50  # 50%

# Calculate missing data percentage
missing_percent = df_daily.isnull().mean()

# Remove columns with too much missing data
columns_to_drop = missing_percent[missing_percent > missing_threshold].index
df_cleaned = df_daily.drop(columns=columns_to_drop)

print(f"Removed columns due to excessive missing data: {list(columns_to_drop)}")

Removed columns due to excessive missing data: ['pressure', 'wind_min_past1h']


In [12]:
df_cleaned.head() # Show first rows

parameterId,temp_min_past1h,temp_max_past1h,temp_mean_past1h,temp_grass_mean_past1h,temp_soil_min_past1h,temp_soil_max_past1h,temp_soil_mean_past1h,humidity_past1h,wind_dir_past1h,wind_gust_always_past1h,wind_speed_past1h,precip_past1h,precip_dur_past1h,radia_glob_past1h,sun_last1h_glob
time,,,,,,,,,,,,,,,
2002-01-01 00:00:00+00:00,-9.3,6.4,1.250000,-0.062500,-0.2,0.1,0.037500,88.708333,281.666667,NaN,6.995833,2.2,34.0,271.0,0.0
2002-01-02 00:00:00+00:00,-6.3,6.4,-0.837500,-1.970833,-0.1,0.0,-0.033333,77.583333,110.208333,NaN,2.695833,0.0,0.0,805.0,328.0
2002-01-03 00:00:00+00:00,-7.6,-1.0,-5.166667,-6.937500,-0.2,0.0,-0.100000,87.458333,158.625000,NaN,1.195833,0.0,0.0,739.0,278.5
2002-01-04 00:00:00+00:00,-9.3,-1.3,-6.025000,-7.950000,-0.7,-0.2,-0.333333,80.916667,221.708333,NaN,0.987500,0.0,0.0,876.0,355.0
2002-01-05 00:00:00+00:00,-10.6,3.7,-3.466667,-4.337500,-1.1,-0.4,-0.754167,92.458333,239.750000,NaN,1.854167,0.1,1.0,659.0,268.5


In [25]:
# Step 1: Identify missing values before processing
missing_before = df_cleaned.isnull().sum()
print(f"Missing values before processing:\n{missing_before[missing_before > 0]}\n")

# Step 2: Apply linear interpolation for gaps up to 12 hours
df_cleaned = df_cleaned.interpolate(method='linear', limit=12, limit_direction='both')

# Step 3: Add helper columns for seasonal mean imputation
df_cleaned["hour"] = df_cleaned.index.hour
df_cleaned["dayofyear"] = df_cleaned.index.dayofyear
df_cleaned["month"] = df_cleaned.index.month
df_cleaned["day"] = df_cleaned.index.day
df_cleaned["year"] = df_cleaned.index.year  # Keep track of leap years


# Step 5: Fill remaining missing values using seasonal means, handling leap years properly
df_cleaned = df_cleaned.groupby(["month", "day", "hour"]).transform(lambda x: x.fillna(x.mean()))

# Step 6: Remove helper columns after filling
df_cleaned.drop(columns=["hour", "dayofyear", "month", "day", "year"], errors="ignore", inplace=True)

# Step 7: Save the cleaned dataset to a new CSV file
df_cleaned.to_csv("dmi_weather_data_cleaned.csv")

# Step 8: Identify missing values after processing
missing_after = df_cleaned.isnull().sum()
print(f"Missing values after processing:\n{missing_after[missing_after > 0]}")
print("\nData cleaning complete!")

df_cleaned.head() # Show first rows

Missing values before processing:
Series([], dtype: int64)

Missing values after processing:
Series([], dtype: int64)

Data cleaning complete!


parameterId,temp_min_past1h,temp_max_past1h,temp_mean_past1h,temp_grass_mean_past1h,temp_soil_min_past1h,temp_soil_max_past1h,temp_soil_mean_past1h,humidity_past1h,wind_dir_past1h,wind_gust_always_past1h,wind_speed_past1h,precip_past1h,precip_dur_past1h,radia_glob_past1h,sun_last1h_glob
time,,,,,,,,,,,,,,,
2002-01-01 00:00:00+00:00,-9.3,6.4,1.250000,-0.062500,-0.2,0.1,0.037500,88.708333,281.666667,10.880,6.995833,2.2,34.0,271.0,0.0
2002-01-02 00:00:00+00:00,-6.3,6.4,-0.837500,-1.970833,-0.1,0.0,-0.033333,77.583333,110.208333,10.995,2.695833,0.0,0.0,805.0,328.0
2002-01-03 00:00:00+00:00,-7.6,-1.0,-5.166667,-6.937500,-0.2,0.0,-0.100000,87.458333,158.625000,12.765,1.195833,0.0,0.0,739.0,278.5
2002-01-04 00:00:00+00:00,-9.3,-1.3,-6.025000,-7.950000,-0.7,-0.2,-0.333333,80.916667,221.708333,13.415,0.987500,0.0,0.0,876.0,355.0
2002-01-05 00:00:00+00:00,-10.6,3.7,-3.466667,-4.337500,-1.1,-0.4,-0.754167,92.458333,239.750000,11.115,1.854167,0.1,1.0,659.0,268.5


In [26]:
import pandas as pd

# # Load cleaned weather data
# file_path = "Cleaned data etc./dmi_weather_data_cleaned.csv"
# df = pd.read_csv(file_path, header=0, parse_dates=["time"], index_col="time")

# Ensure `time` is datetime and remove any timezone info
df_cleaned.index = pd.to_datetime(df_cleaned.index).tz_localize(None)

# Load inflow data (KarupBro)
file_path_korup_parken = 'Raw data/Bygholm_Kørup_Parken_VNF_døgn_1975-2023.csv'
korup_parken_df = pd.read_csv(file_path_korup_parken, decimal=",", encoding="ISO-8859-1", delimiter=";")

# Clean column names
korup_parken_df.columns = korup_parken_df.columns.str.strip()
korup_parken_df.columns = ["timestamp", "KarupBro", "BygholmPark"]

# Convert timestamp to datetime (ensure it's properly formatted)
korup_parken_df["timestamp"] = pd.to_datetime(korup_parken_df["timestamp"], format="%d-%m-%Y")

# Set timestamp as index to match `df`
korup_parken_df.set_index("timestamp", inplace=True)

# Drop unnecessary column
korup_parken_df.drop(columns=["BygholmPark"], inplace=True)

# Merge data on the index instead of the column
merged_df = df_cleaned.join(korup_parken_df, how="left")

# Check result
merged_df.tail()

,temp_min_past1h,temp_max_past1h,temp_mean_past1h,temp_grass_mean_past1h,temp_soil_min_past1h,temp_soil_max_past1h,temp_soil_mean_past1h,humidity_past1h,wind_dir_past1h,wind_gust_always_past1h,wind_speed_past1h,precip_past1h,precip_dur_past1h,radia_glob_past1h,sun_last1h_glob,KarupBro
time,,,,,,,,,,,,,,,,
2023-12-20,0.8,4.4,3.141667,2.316667,5.1,5.4,5.241667,88.583333,252.000000,12.2,2.541667,0.4,4.0,349.0,111.0,4448.18
2023-12-21,1.4,8.7,5.316667,4.637500,5.0,5.3,5.141667,86.083333,254.666667,22.5,5.329167,19.9,159.0,270.0,29.5,5602.14
2023-12-22,0.1,4.0,2.395833,1.129167,4.0,5.1,4.533333,66.000000,293.416667,19.1,5.020833,0.6,3.0,749.0,294.5,6636.60
2023-12-23,-1.6,3.3,0.833333,-0.250000,3.6,4.1,3.754167,88.000000,264.000000,10.5,2.379167,0.8,4.0,589.0,247.0,5809.15
2023-12-24,-0.3,2.1,1.120000,0.700000,3.4,3.6,3.520000,97.700000,136.500000,6.6,1.500000,5.0,38.0,9.0,0.0,5197.78


In [27]:
merged_df.rename(columns={
    'temp_min_past1h': 'temp_min_daily',
    'temp_max_past1h': 'temp_max_daily',
    'temp_mean_past1h': 'temp_mean_daily',
    'temp_grass_mean_past1h': 'temp_grass_mean_daily',
    'temp_soil_min_past1h': 'temp_soil_min_daily',
    'temp_soil_max_past1h': 'temp_soil_max_daily',
    'temp_soil_mean_past1h': 'temp_soil_mean_daily',
    'humidity_past1h': 'humidity_daily',
    'wind_dir_past1h': 'wind_direction_daily',
    'wind_speed_past1h': 'wind_speed_daily',
    'wind_gust_always_past1h': 'wind_gust_max_daily',
    'precip_past1h': 'precip_total_daily',
    'precip_dur_past1h': 'precip_duration_daily',
    'radia_glob_past1h': 'radiation_global_daily',
    'sun_last1h_glob': 'sunshine_duration_daily',
    'KarupBro': 'KarupBro',  # Assuming this is a station or location name
}, inplace=True)

# Check for missing values
merged_df.isnull().sum()

temp_min_daily             0
temp_max_daily             0
temp_mean_daily            0
temp_grass_mean_daily      0
temp_soil_min_daily        0
temp_soil_max_daily        0
temp_soil_mean_daily       0
humidity_daily             0
wind_direction_daily       0
wind_gust_max_daily        0
wind_speed_daily           0
precip_total_daily         0
precip_duration_daily      0
radiation_global_daily     0
sunshine_duration_daily    0
KarupBro                   0
dtype: int64

In [28]:
merged_df.to_csv("data_cleaned_and_interpolated.csv")